In [9]:
print("hello world")

hello world


In [10]:
import gtfs_kit

In [11]:
import gtfs_kit

In [12]:
import gtfs_kit as gk

In [ ]:
feed = gk.read_feed("gtfs_subway.zip", dist_units="m")

In [ ]:
feed = gk.read_feed("gtfs_subway.zip", dist_units="m")

In [ ]:
feed.describe()

,indicator,value
0,agencies,[MTA New York City Transit]
1,timezone,America/New_York
2,start_date,20251102
3,end_date,20251207
4,num_routes,29
5,num_trips,20298
6,num_stops,1488
7,num_shapes,311
8,sample_date,20251106
9,num_routes_active_on_sample_date,29


In [ ]:
import gtfs_kit as gk
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def calculate_headways(feed, route_id, direction_id=None, service_date=None):
    """
    Calculate train headways for each hour of the day for a specific route.
    
    Parameters:
    - feed: gtfs_kit Feed object
    - route_id: Route ID (e.g., '1', 'A', 'L')
    - direction_id: 0 or 1 for direction (optional, None includes both)
    - service_date: Date string 'YYYYMMDD' (optional, uses a typical weekday if None)
    """
    
    # Get stop times for the route
    route_trips = feed.trips[feed.trips['route_id'] == route_id].copy()
    
    if route_trips.empty:
        print(f"No trips found for route {route_id}")
        return None
    
    # Filter by direction if specified
    if direction_id is not None:
        route_trips = route_trips[route_trips['direction_id'] == direction_id]
    
    # Filter by service date if specified
    if service_date:
        # Get active services for the date
        active_services = feed.get_services(service_date)
        route_trips = route_trips[route_trips['service_id'].isin(active_services)]
    
    # Get stop times for these trips
    stop_times = feed.stop_times[feed.stop_times['trip_id'].isin(route_trips['trip_id'])].copy()
    
    # Get the first stop for each trip (as a proxy for train departure time)
    first_stops = stop_times.loc[stop_times.groupby('trip_id')['stop_sequence'].idxmin()]
    
    # Convert time strings to seconds since midnight
    first_stops['departure_seconds'] = first_stops['departure_time'].apply(time_to_seconds)
    
    # Sort by departure time
    first_stops = first_stops.sort_values('departure_seconds')
    
    # Calculate headways (difference between consecutive trains)
    first_stops['headway_minutes'] = first_stops['departure_seconds'].diff() / 60
    
    # Add hour of day
    first_stops['hour'] = (first_stops['departure_seconds'] // 3600) % 24
    
    # Calculate statistics by hour
    hourly_stats = first_stops.groupby('hour')['headway_minutes'].agg([
        ('count', 'count'),
        ('mean_headway', 'mean'),
        ('median_headway', 'median'),
        ('min_headway', 'min'),
        ('max_headway', 'max'),
        ('std_headway', 'std')
    ]).round(2)
    
    return hourly_stats, first_stops

def time_to_seconds(time_str):
    """Convert GTFS time string (HH:MM:SS) to seconds since midnight."""
    parts = time_str.split(':')
    hours = int(parts[0])
    minutes = int(parts[1])
    seconds = int(parts[2])
    return hours * 3600 + minutes * 60 + seconds

# Example usage
if __name__ == "__main__":
    # Load GTFS feed
    # Download from: https://api.mta.info/gtfs/subway/[feed_id]
    feed_path = "gtfs_subway.zip"  # GTFS file in repository root
    
    print("Loading GTFS feed...")
    feed = gk.read_feed(feed_path, dist_units='mi')
    
    # Show available routes
    print("\nAvailable routes:")
    print(feed.routes[['route_id', 'route_short_name', 'route_long_name']])
    
    # Calculate headways for a specific route (e.g., the 1 train)
    route_id = '1'  # Change this to your desired route
    
    print(f"\nCalculating headways for route {route_id}...")
    hourly_stats, detailed_data = calculate_headways(feed, route_id)
    
    if hourly_stats is not None:
        print(f"\nHourly Headway Statistics for Route {route_id}:")
        print(hourly_stats)
        
        # Optional: Save to CSV
        # hourly_stats.to_csv(f'route_{route_id}_headways.csv')
        # print(f"\nResults saved to route_{route_id}_headways.csv")

In [ ]:
calculate_headways(feed, "A", 1)

(      count  mean_headway  median_headway  min_headway  max_headway  \
 hour                                                                  
 0        10         10.20            0.50          0.0         61.0   
 1        18          3.53            0.25          0.0         20.0   
 2        18          3.33            1.50          0.0         13.5   
 3        18          3.33            1.50          0.0         13.5   
 4        18          3.33            1.50          0.0         13.5   
 5        19          3.24            2.50          0.0         13.5   
 6        28          2.32            2.00          0.0          8.5   
 7        22          2.73            2.00          0.0          7.0   
 8        21          2.86            2.00          0.0          7.0   
 9        21          2.86            2.50          0.0          7.5   
 10       19          3.16            2.50          0.0          8.0   
 11       20          3.00            1.50          0.0         

In [2]:
import gtfs_kit as gk
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict


def get_line_headways_by_hour(feed, route_id, direction_id=None, service_id=None):
    """
    Calculate train headways for each hour of the day for a particular line.
    
    Parameters:
    -----------
    feed : gtfs_kit.Feed
        A GTFS feed object loaded with gtfs_kit
    route_id : str
        The route ID for the line (e.g., '1', 'A', 'L')
    direction_id : int, optional
        Direction ID (0 or 1) to filter by direction. If None, includes both directions.
    service_id : str, optional
        Service ID to filter by (e.g., weekday, weekend service). If None, uses all services.
    
    Returns:
    --------
    dict
        Dictionary with hours (0-23) as keys and lists of headways (in minutes) as values
    """
    
    # Get stop times for the specified route
    trips = feed.trips[feed.trips['route_id'] == route_id].copy()
    
    # Filter by direction if specified
    if direction_id is not None:
        trips = trips[trips['direction_id'] == direction_id]
    
    # Filter by service_id if specified
    if service_id is not None:
        trips = trips[trips['service_id'] == service_id]
    
    if trips.empty:
        print(f"No trips found for route {route_id}")
        return {}
    
    # Get stop times for these trips
    stop_times = feed.stop_times[feed.stop_times['trip_id'].isin(trips['trip_id'])].copy()
    
    # Group by trip and get the first stop time for each trip (to represent departure time)
    first_stops = stop_times.groupby('trip_id').first().reset_index()
    
    # Merge with trips to get all trip information
    trip_departures = first_stops.merge(trips[['trip_id', 'route_id', 'direction_id', 'service_id']], 
                                        on='trip_id')
    
    # Convert departure_time to datetime for easier manipulation
    # Note: GTFS times can go beyond 24:00:00 for trips after midnight
    def parse_gtfs_time(time_str):
        """Parse GTFS time format (which can exceed 24 hours)"""
        parts = time_str.split(':')
        hours = int(parts[0])
        minutes = int(parts[1])
        seconds = int(parts[2])
        return hours, minutes, seconds
    
    # Extract hour from departure time
    trip_departures['hour'] = trip_departures['departure_time'].apply(
        lambda x: parse_gtfs_time(x)[0] % 24
    )
    
    # Convert to total seconds for easier headway calculation
    trip_departures['departure_seconds'] = trip_departures['departure_time'].apply(
        lambda x: sum([parse_gtfs_time(x)[0] * 3600, 
                      parse_gtfs_time(x)[1] * 60, 
                      parse_gtfs_time(x)[2]])
    )
    
    # Sort by departure time
    trip_departures = trip_departures.sort_values('departure_seconds')
    
    # Calculate headways by hour
    headways_by_hour = defaultdict(list)
    
    for hour in range(24):
        # Get all departures in this hour
        hour_departures = trip_departures[trip_departures['hour'] == hour].copy()
        hour_departures = hour_departures.sort_values('departure_seconds')
        
        # Calculate headways between consecutive trains
        if len(hour_departures) > 1:
            departure_times = hour_departures['departure_seconds'].values
            headways = []
            
            for i in range(1, len(departure_times)):
                headway_seconds = departure_times[i] - departure_times[i-1]
                headway_minutes = headway_seconds / 60.0
                headways.append(headway_minutes)
            
            headways_by_hour[hour] = headways
    
    return dict(headways_by_hour)


def display_headway_summary(headways_by_hour):
    """
    Display a summary of headways by hour with statistics.
    
    Parameters:
    -----------
    headways_by_hour : dict
        Dictionary returned by get_line_headways_by_hour
    """
    print(f"{'Hour':<6} {'# Trains':<10} {'Avg Headway':<15} {'Min Headway':<15} {'Max Headway':<15}")
    print("-" * 70)
    
    for hour in sorted(headways_by_hour.keys()):
        headways = headways_by_hour[hour]
        if headways:
            avg_hw = sum(headways) / len(headways)
            min_hw = min(headways)
            max_hw = max(headways)
            num_trains = len(headways) + 1  # +1 because n trains have n-1 headways
            
            print(f"{hour:02d}:00  {num_trains:<10} {avg_hw:<15.2f} {min_hw:<15.2f} {max_hw:<15.2f}")
        else:
            print(f"{hour:02d}:00  {'0':<10} {'-':<15} {'-':<15} {'-':<15}")


# Example usage:
if __name__ == "__main__":
    # Load your GTFS feed
    # feed = gk.read_feed("path/to/your/gtfs.zip", dist_units='km')
    
    # Example: Get headways for the 1 train
    # headways = get_line_headways_by_hour(feed, route_id='1')
    # display_headway_summary(headways)
    
    # Example: Get headways for a specific direction and service
    # headways = get_line_headways_by_hour(feed, route_id='A', direction_id=0, service_id='A20240101WKD')
    # display_headway_summary(headways)
    
    print("Functions defined. Load your GTFS feed and call get_line_headways_by_hour() to use.")

Functions defined. Load your GTFS feed and call get_line_headways_by_hour() to use.


In [ ]:
display_headway_summary(get_line_headways_by_hour(feed, "A", 1, "Weekday"))

Hour   # Trains   Avg Headway     Min Headway     Max Headway    
----------------------------------------------------------------------
00:00  3          19.00           18.00           20.00          
01:00  6          8.70            3.50            16.50          
02:00  6          8.70            3.50            16.50          
03:00  6          8.70            3.50            16.50          
04:00  6          8.70            3.50            16.50          
05:00  7          7.58            1.00            13.00          
06:00  11         5.50            2.00            9.50           
07:00  9          6.62            1.00            9.00           
08:00  10         5.94            4.50            8.50           
09:00  10         6.17            3.50            7.50           
10:00  7          8.00            8.00            8.00           
11:00  8          8.00            7.00            8.50           
12:00  7          8.00            8.00            8.00           
13:00

In [ ]:
display_headway_summary(get_line_headways_by_hour(feed, "C", 1, "Weekday"))

Hour   # Trains   Avg Headway     Min Headway     Max Headway    
----------------------------------------------------------------------
05:00  4          13.67           10.50           15.50          
06:00  6          10.50           9.50            11.00          
07:00  6          10.10           9.50            11.50          
08:00  6          9.00            8.00            9.50           
09:00  7          9.08            7.50            10.50          
10:00  8          8.00            7.00            9.50           
11:00  7          8.17            7.00            9.00           
12:00  8          7.79            7.50            8.00           
13:00  7          8.25            7.50            9.00           
14:00  7          8.58            6.50            10.00          
15:00  6          9.70            9.00            11.00          
16:00  6          9.70            8.00            11.50          
17:00  6          9.80            8.50            11.50          
18:00

In [3]:
display_headway_summary(get_line_headways_by_hour(feed, "W", 1, "Weekday"))

NameError: name 'feed' is not defined